In [11]:
import requests
import pandas as pd
import time
from IPython.display import JSON

In [12]:
# keys
api_key = "xxxxxxx"
channel_id = "UCCKlp1JI9Yg3-cUjKPdD3mw" # magic ingredients

In [13]:
#url = "https://www.googleapis.com/youtube/v3/videos?id="+"76K8D_cwcvU"+"&part=statistics&key="+ api_key
#response = requests.get(url).json()

In [14]:
#JSON(response['items'][0])

In [15]:
def get_video_details(video_id):

    #collecting view, like, dislike, comment counts
    video_stats_url = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=statistics&key="+ api_key
    response_video_stats = requests.get(video_stats_url).json()
    
    JSON(response_video_stats['items'][0]['statistics'])
    try:
        view_count = response_video_stats['items'][0]['statistics']['viewCount']
    except:
        view_count = None
    try:
        like_count = response_video_stats['items'][0]['statistics']['likeCount']
    except:
        like_count = None
    try:
        dislike_count = response_video_stats['items'][0]['statistics']['dislikeCount']
    except:
        dislike_count = None
    try:
        comment_count = response_video_stats['items'][0]['statistics']['commentCount']
    except:
        comment_count = None
        
    return view_count, like_count, dislike_count, comment_count

In [16]:
def get_videos(df): 
    pageToken = "" 
    while True: 
        url = "https://www.googleapis.com/youtube/v3/search?key="+api_key+"&channelId="+channel_id+"&part=snippet,id&order=date&maxResults=50&"+pageToken

        response = requests.get(url).json()
        time.sleep(1) #give it a second before starting the for loop
        for video in response['items']:
            if video['id']['kind'] == "youtube#video":
                video_id = video['id']['videoId']
                channel_title = video['snippet']['channelTitle']
                video_title = video['snippet']['title']
                video_title = str(video_title).replace("&","")
                upload_date = video['snippet']['publishedAt']
                upload_date = str(upload_date).split("T")[0]
                view_count, like_count, dislike_count, comment_count = get_video_details(video_id)

                video_item = pd.DataFrame(data={'video_id':video_id, 'channel_title':channel_title, 'video_title':video_title, 'upload_date':upload_date,
                                                'view_count':view_count, 'like_count':like_count, 'dislike_count':dislike_count, 'comment_count':comment_count},
                                         index =['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h'])
                df = pd.concat([df, video_item], ignore_index=True, verify_integrity=False)
        try:
            if response['nextPageToken'] != None: #if none, it means it reached the last page and break out of it
                pageToken = "pageToken=" + response['nextPageToken']
        except:
            break
    return df

In [17]:
#build our dataframe
video_df = pd.DataFrame(columns=["video_id", "channel_title", "video_title", "upload_date", "view_count", "like_count", "dislike_count", "comment_count"]);
video_df = get_videos(video_df);

In [18]:
video_df = video_df.drop_duplicates()
video_df.shape

(302, 8)

In [19]:
video_df.head(5)

,video_id,channel_title,video_title,upload_date,view_count,like_count,dislike_count,comment_count
0,76K8D_cwcvU,小高姐的 Magic Ingredients,素五鲜水饺,2023-01-21,80071,4444,None,399
8,FWG_1oZTcS0,小高姐的 Magic Ingredients,蒜蓉虾仁 Garlic Shrimp,2023-01-13,129069,5227,None,369
16,JpIymnVO5eY,小高姐的 Magic Ingredients,搓麻食,2022-12-21,107346,4907,None,485
24,1q_I5IHdrdE,小高姐的 Magic Ingredients,灌蛋油条 油条简单做法,2022-11-03,170599,6627,None,362
32,8P5TlolEPL0,小高姐的 Magic Ingredients,麻酱烧饼,2022-10-20,169710,7426,None,383


In [20]:
video_df.to_csv('Magic_Ingredients_Videos_Stats.csv', index=False)